### Explainer Experiment Demo

Below we show some examples on how you can use the **Experiment_Explainers** class. 

You can set distribution=True to enable distribution plots while running the experiments. When the experiment is completed, result table is saved in /experiment_plots.

In [1]:
import warnings
from cornac.datasets import goodreads
from cornac.experiment.experiment_explainers import Experiment_Explainers
from cornac.models import EMF, ALS, FMRec, MTER, EFM, NEMF
from cornac.explainer import Exp_EFM, Exp_LIMERS, Exp_MTER, Exp_ALS, Exp_PHI4MF, Exp_SU4EMF
from cornac.datasets.goodreads import prepare_data
from cornac.metrics_explainer import Metric_Exp_PSPNFNS as PSPN, Metric_Exp_PGF as PGF, Metric_Exp_MEP as MEP, Metric_Exp_EnDCG as EnDCG
from cornac.metrics_explainer import Metric_Exp_DIV as DIV, Metric_Exp_FPR as FPR, Metric_Exp_FA as FA, Metric_Exp_RA as RA

warnings.filterwarnings('ignore') 

/home/linan/miniconda3/envs/cornac/lib/python3.11/site-packages/tensorflow/python/debug/cli/debugger_cli_common.py:19: DeprecationWarning: module 'sre_constants' is deprecated
  import sre_constants


Experiment 1: run PSPNFNS, FDIV and FPR on (FM, Exp_LIMERS) pair using data from goodreads_limers

In [4]:
rs = prepare_data(data_name="goodreads_limers", test_size=0.2, dense=True, item=True, user=True, sample_size=1, seed=21)
fm = FMRec()
limers = Exp_LIMERS(rec_model=fm, dataset=rs.train_set)
pspnfns = PSPN()
fdiv = DIV()
fpr = FPR()

models = [(fm, limers)]
metrics = [pspnfns, fdiv, fpr]
experiment = Experiment_Explainers(eval_method=rs, models=models, metrics=metrics,distribution=True, rec_k=10, feature_k=10, eval_train=True)
experiment.run()

INFO:cornac.experiment.experiment_explainers:Start training Recommender FMRec...


Creating validation dataset of 0.01 of training for adaptive regularization
-- Epoch 1
Training MSE: 0.93451
-- Epoch 2
Training MSE: 0.79441
-- Epoch 3
Training MSE: 0.75649
-- Epoch 4
Training MSE: 0.73454
-- Epoch 5
Training MSE: 0.71571
-- Epoch 6
Training MSE: 0.70002
-- Epoch 7


INFO:cornac.experiment.experiment_explainers:*****Start evaluating model-explainer: 'FMRec:Exp_LIMERS'...
INFO:cornac.experiment.experiment_explainers:Step 1/3: Recommender FMRec creates recommendations


Training MSE: 0.69143
-- Epoch 8
Training MSE: 0.67739
-- Epoch 9
Training MSE: 0.66803
-- Epoch 10
Training MSE: 0.65927


INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer Exp_LIMERS create explanation for all recommendations
Computing explanations: 100%|██████████| 1000/1000 [00:14<00:00, 67.14it/s]
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_PSPNFNS starts evaluation...
INFO:cornac.experiment.experiment_explainers:self.current_rec: FMRec, self.current_exp: Exp_LIMERS
Re-evaluate after features removal... : 100%|██████████| 1000/1000 [00:16<00:00, 60.11it/s]
INFO:cornac.experiment.experiment_explainers:Result: Probability of Necessity: 0.46; Probability of Sufficiency: 0.317; Harmonic Mean: 0.37534105534105533
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_DIV starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Feature diversity: 0.45654413858288023
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_FPR starts evaluation...
ERROR:cornac.experiment.experiment_explainers:Metric Metric_Exp_FPR d

Experiment 2: run PSPN, FDIV and FPR on (EFM, Exp_EFM) and (MTER, Exp_MTER) pairs using data from goodreads

In [5]:
rs = prepare_data(data_name="goodreads", test_size=0.2, dense=True, item=True, user=True, sample_size=1, seed=21)
efm = EFM(max_iter=20, num_explicit_factors=50, num_latent_factors=50, num_most_cared_aspects=50, alpha=0.85,lambda_h=0.1, lambda_u=0.001, lambda_v=0.01, lambda_x=1.0, lambda_y= 0.1)
efm_exp = Exp_EFM(rec_model=efm, dataset=rs.train_set)
mter = MTER(max_iter=20, n_aspect_factors=8, n_item_factors=5, n_opinion_factors= 5, n_user_factors= 10, lambda_bpr=10, lambda_reg= 10, n_bpr_samples=1000, n_element_samples=50)
mter_exp = Exp_MTER(rec_model=mter, dataset=rs.train_set)

pspnfns = PSPN()
fdiv = DIV()
fpr = FPR()

models = [(efm, efm_exp), (mter, mter_exp)]
metrics = [fdiv, fpr, pspnfns]
experiment = Experiment_Explainers(eval_method=rs, models=models, metrics=metrics,rec_k=10, distribution=False, feature_k=10, eval_train=True)
experiment.run()

INFO:cornac.experiment.experiment_explainers:Start training Recommender EFM...
INFO:cornac.experiment.experiment_explainers:*****Start evaluating model-explainer: 'EFM:Exp_EFM'...
INFO:cornac.experiment.experiment_explainers:Step 1/3: Recommender EFM creates recommendations
INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer Exp_EFM create explanation for all recommendations
Computing explanations: 100%|██████████| 1000/1000 [00:00<00:00, 9238.31it/s]
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_DIV starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Feature diversity: 0.5131798465131876
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_FPR starts evaluation...
Start evaluation... :  20%|██        | 1440/7200 [00:00<00:02, 2327.50it/s] 
INFO:cornac.experiment.experiment_explainers:Result: Feature Precision: 0.37676609105180536; Feature Recall: 0.06329162981303943; Harmonic Mean: 0.09951375281973754
IN

Experiment 3: run FDIV, PGF, MEP and EnDCG on (ALS, Exp_ALS), (EMF, Exp_PHI4MF) and (NEMF, Exp_SU4EMF) pairs using data from goodreads

In [2]:
rs_dense = prepare_data(data_name="goodreads_uir_1000", test_size=0, dense=True, item=True, user=True, sample_size=1, seed=21)
emf = EMF(k=10, max_iter=500, learning_rate=0.001, lambda_reg=0.1, explain_reg=0.01, verbose=True, seed=6, num_threads=6, early_stop=True)
nemf = NEMF(k=10, max_iter=500, learning_rate=0.001, lambda_reg=0.1, explain_reg=0.01, novel_reg=1, verbose=True, seed=6, num_threads=6, early_stop=True)
als = ALS(k=10, max_iter=500, lambda_reg=0.001, alpha=1, verbose=True, seed=6)
als_exp = Exp_ALS(rec_model=als, dataset=rs_dense.train_set)
emf_exp = Exp_PHI4MF(rec_model=emf, dataset=rs_dense.train_set)
nemf_exp = Exp_SU4EMF(rec_model=nemf, dataset=rs_dense.train_set)
fdiv = DIV()
pgf = PGF()
mep = MEP()
endcg = EnDCG()
models = [(als, als_exp), (emf, emf_exp), (nemf, nemf_exp)]
metrics = [fdiv, pgf, mep, endcg]
experiment = Experiment_Explainers(eval_method=rs_dense, models=models, metrics=metrics,distribution=False, rec_k=10, feature_k=10, eval_train=True)
experiment.run()

INFO:cornac.experiment.experiment_explainers:Start training Recommender ALS...


  0%|          | 0/500 [00:00<?, ?it/s]

INFO:implicit:Final training loss 0.0284
INFO:cornac.experiment.experiment_explainers:*****Start evaluating model-explainer: 'ALS:Exp_ALS'...
INFO:cornac.experiment.experiment_explainers:Step 1/3: Recommender ALS creates recommendations
INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer Exp_ALS create explanation for all recommendations


Computing explanations:   0%|          | 0/1000 [00:00<?, ?it/s]

INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_DIV starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Feature diversity: 0.14335795795775053
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_PGF starts evaluation...
100%|██████████| 100/100 [00:00<00:00, 1306.53it/s]
INFO:cornac.experiment.experiment_explainers:Result: Metric_Exp_PGF: 0.36180768906387434
ERROR:cornac.experiment.experiment_explainers:Metric Metric_Exp_MEP does not support Exp_ALS.
ERROR:cornac.experiment.experiment_explainers:Metric Metric_Exp_EnDCG does not support Exp_ALS.
INFO:cornac.experiment.experiment_explainers:Start training Recommender EMF...


...

  0%|          | 0/500 [00:00<?, ?it/s]

INFO:cornac.experiment.experiment_explainers:*****Start evaluating model-explainer: 'NEMF:Exp_SU4EMF'...
INFO:cornac.experiment.experiment_explainers:Step 1/3: Recommender NEMF creates recommendations
INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer Exp_SU4EMF create explanation for all recommendations


Optimization finished!


Computing explanations:   0%|          | 0/1000 [00:00<?, ?it/s]

INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_DIV starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Feature diversity: 0.38766852169618093
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_PGF starts evaluation...
100%|██████████| 100/100 [00:00<00:00, 912.11it/s]
INFO:cornac.experiment.experiment_explainers:Result: Metric_Exp_PGF: 0.2339124830812216
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_MEP starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Metric_Exp_MEP: 0.992
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_EnDCG starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Metric_Exp_EnDCG: 0.5730672228840049
INFO:cornac.experiment.experiment_explainers:experiment data: [[0.14335795795775053, 0.36180768906387434, 'N/A', 'N/A', 1.8691916465759277, 4.350937843322754], [0.002826826826826498, 0.08231880964711308, 0.99, 

Experiment 4: Run RA and FA on pairwise models 

In [7]:
rs = prepare_data(data_name="goodreads", test_size=0.2, dense=True, item=True, user=True, sample_size=1, seed=21)
efm = EFM(max_iter=20, num_explicit_factors=50, num_latent_factors=50, num_most_cared_aspects=50, alpha=0.85,lambda_h=0.1, lambda_u=0.001, lambda_v=0.01, lambda_x=1.0, lambda_y= 0.1)
efm_exp = Exp_EFM(rec_model=efm, dataset=rs.train_set)
mter = MTER(max_iter=20, n_aspect_factors=8, n_item_factors=5, n_opinion_factors= 5, n_user_factors= 10, lambda_bpr=10, lambda_reg= 10, n_bpr_samples=1000, n_element_samples=50)
mter_exp = Exp_MTER(rec_model=mter, dataset=rs.train_set)

fa = FA()
ra = RA()
fdiv = DIV()
models = [[(efm, mter), (efm_exp, mter_exp)], (efm, efm_exp), (mter, mter_exp)]
metrics = [fa,ra, fdiv]
experiment = Experiment_Explainers(eval_method=rs, models=models, metrics=metrics,rec_k=10, feature_k=10, eval_train=True)
experiment.run()

INFO:cornac.experiment.experiment_explainers:Start training Recommender1 EFM...
INFO:cornac.experiment.experiment_explainers:Start training Recommender2 MTER...
INFO:cornac.experiment.experiment_explainers:*****Start evaluating model-explainer: 'EFM:Exp_EFM'vs'MTER:Exp_MTER'...
INFO:cornac.experiment.experiment_explainers:Step 1/3: Creates fake recommendations from dataset for common used
INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer1 Exp_EFM create explanation for all recommendations
Computing explanations: 100%|██████████| 100/100 [00:00<00:00, 3468.86it/s]
INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer2 Exp_MTER create explanation for all recommendations
Computing explanations: 100%|██████████| 100/100 [00:00<00:00, 4499.26it/s]
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_FA starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Average Metric_Exp_FA: 0.016666666666666666
INFO:cornac.experiment.expe